## REST API Implementation in Python

In [1]:
import flask
from flask import Flask, request
import json
import sqlite3
from datetime import datetime

app = flask.Flask(__name__)
database='sportsbook'
conn = sqlite3.connect(database,timeout=10,check_same_thread=False) 
c = conn.cursor()

# Creating database tables and loading data
def create_db_tables(database):

    c.execute('''
              CREATE TABLE IF NOT EXISTS sports
              ([sports_id] INTEGER PRIMARY KEY, [sports_name] TEXT, [s_active] INTEGER,
              UNIQUE(sports_id, sports_name, s_active))
              ''')
          
    c.execute('''
              CREATE TABLE IF NOT EXISTS events
              ([event_id] INTEGER PRIMARY KEY, [sports_id] INTEGER, [event_name] TEXT,
               [e_active] INTEGER, [type] TEXT, [status] TEXT, [scheduled_start] TEXT, [actual_start] TEXT,
               UNIQUE(event_id, sports_id, event_name, e_active, type, status, scheduled_start, actual_start))
              ''')

    c.execute('''
              CREATE TABLE IF NOT EXISTS selections
              ([selection_id] INTEGER PRIMARY KEY, [selection_name] TEXT, [event_id] INTEGER, [event_name] TEXT, 
              [price] REAL, [active] INTEGER, [outcome] TEXT,
              UNIQUE(selection_id, selection_name, event_id, event_name, price, active, outcome))
              ''')
    
    c.execute('''
              INSERT OR IGNORE INTO sports (sports_id,sports_name,s_active)

                    VALUES
                    (1,'football',1),
                    (2,'badminton',1),
                    (3,'basketball',1),
                    (4,'tennis',1),
                    (5,'hockey',1),
                    (6,'volleyball',1),
                    (7,'golf',1),
                    (8,'baseball',1),
                    (9,'cricket',1),
                    (10,'wrestling',1)

              ''')

    c.execute('''
              INSERT OR IGNORE INTO events (event_id,sports_id,event_name,e_active,type,
              status,scheduled_start,actual_start)

                    VALUES
                    (1,1,'European_Cup',1,'preplay','started','2022-01-21 22:38:31','2022-01-21 22:38:31'),
                    (2,1,'FIFA_World_Cup',1,'preplay','pending','2022-01-22 22:38:31','2022-01-22 22:38:31'),
                    (3,3,'World_Cup',0,'inplay','ended','2022-01-23 22:38:31','2022-01-23 22:38:31'),
                    (4,3,'NBA_finals',1,'inplay','pending','2022-01-24 22:38:31','2022-01-24 22:38:31'),
                    (5,6,'Womens_World_Cup',1,'preplay','pending','2022-01-25 22:38:31','2022-01-25 22:38:31'),
                    (6,6,'Olympics',1,'inplay','pending','2022-01-25 22:38:31','2022-01-25 22:38:31'),
                    (7,6,'World_Championship',1,'preplay','pending','2022-01-25 22:38:31','2022-01-25 22:38:31'),
                    (8,9,'IPL',0,'preplay','pending','2022-01-25 22:38:31','2022-01-25 22:38:31'),
                    (9,9,'World_Cup',0,'inplay','pending','2022-01-25 22:38:31','2022-01-25 22:38:31'),
                    (10,9,'Test_Match',0,'preplay','pending','2022-01-25 22:38:31','2022-01-25 22:38:31')

              ''')

    c.execute('''
              INSERT OR IGNORE INTO selections (selection_id,selection_name,event_id,event_name,price,
              active,outcome)

                    VALUES
                    (1,'s1',1,'European_Cup',100.12,1,'win'),
                    (2,'s2',1,'European_Cup',150.00,1,'lose'),
                    (3,'s3',2,'FIFA_World_Cup',200.00,0,'win'),
                    (4,'s4',3,'World_Cup',200.00,1,'unsettled'),
                    (5,'s5',4,'NBA_finals',300.00,1,'win'),
                    (6,'s6',5,'Womens_World_Cup',500.00,0,'lose'),
                    (7,'s7',8,'IPL-AusvsNz',500.00,0,'lose'),
                    (8,'s8',8,'IPL-SAvsInd',400.00,0,'void'),
                    (9,'s9',8,'IPL-KenyavsNigeria',100.00,0,'void'),
                    (10,'s10',10,'Test-1',1000.00,0,'void')
    

              ''')

create_db_tables(database)
  
# Function to update active status of sports and events tables
# If all the selections of a particular event are inactive, the event becomes inactive
# And if all the events of a sport are inactive, the sport becomes inactive

def update_active():
    c.execute('''
          UPDATE events SET e_active=CASE 
          WHEN event_id IN (SELECT DISTINCT a.event_id FROM events a INNER JOIN selections b ON a.event_id==b.event_id WHERE b.active=1) THEN 1
          ELSE 0
          END
          ''')
    c.execute('''
          UPDATE sports SET s_active=CASE 
          WHEN sports_id IN (SELECT DISTINCT a.sports_id FROM sports a INNER JOIN events b ON a.sports_id==b.sports_id WHERE b.e_active=1) THEN 1
          ELSE 0
          END
          ''')

update_active()

# REST API Methods

# Home page
@app.route('/', methods=['GET'])
def home_page():
    title=f"Sports Book : Web App for managing sports, events and selections"
    return title

# Method to GET all sports
@app.route('/sports', methods=['GET'])
def get_sports():
    json_list = []
    conn.row_factory = sqlite3.Row
    c.execute('SELECT * FROM sports;')
    sport=c.fetchall()    
    for row in sport:
        json_dict = {'sports_id': row[0], 'sports_name': row[1], 's_active': row[2]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    s = json.loads(x)
    json_output = {'sports': s}
    return json_output

# Method to GET individual sports by sports name
@app.route('/sports/<string:name>', methods=['GET'])
def get_sport_name(name):
    json_list = []
    conn.row_factory = sqlite3.Row
    c.execute('SELECT * FROM sports WHERE sports_name==?;',(name,))
    sport=c.fetchall()    
    for row in sport:
        json_dict = {'sports_id': row[0], 'sports_name': row[1], 's_active': row[2]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    s = json.loads(x)
    json_output = {'sports': s}
    return json_output

# Method to Create/POST sports
@app.route('/sports', methods=['POST'])
def create_sports():
    sports=request.get_json()
    c.execute('''INSERT OR IGNORE INTO sports (sports_id, sports_name, s_active) 
                    VALUES (?, ?, ?)''', (sports['sports_id'],   
                    sports['sports_name'], sports['s_active']) )
    conn.commit()
    update_active()
    return "success"

# Method to GET all "active" events
@app.route('/events', methods=['GET'])
def get_active_events():
    json_list = []              
    conn.row_factory = sqlite3.Row
    #cur = conn.cursor()
    c.execute('SELECT * FROM events WHERE e_active==1;')
    event=c.fetchall()    
    for row in event:
        json_dict = {'event_id':row[0],'sports_id':row[1],'event_name': row[2], 'e_active': row[3],
                     'type':row[4],'status ': row[5],'scheduled_start':row[6],'actual_start': row[7]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    s = json.loads(x)
    json_output = {'events': s}
    return json_output

# Method to Update/PUT events
# Updating status of a particular event to "started" will update its "actual_start" to reflect current date & time
@app.route('/events',  methods=['PUT'])
def update_events():
    event=request.get_json()
    if event["status"]=="started":
        now = str(datetime.now())
        c.execute('''UPDATE events SET status=?,actual_start=?
                     WHERE event_name=? ''',(event["status"],now,event["event_name"]))
    else:
        c.execute('''UPDATE events SET status=? WHERE event_name=? ''',(event["status"],event["event_name"]))
        
    conn.commit()
    return "success"

# Method to GET all sports,events & selections that has started and has type as "preplay"
@app.route('/started', methods=['GET'])
def get_started():
    json_list = []              
    conn.row_factory = sqlite3.Row
    c.execute('''
              SELECT a.sports_name,b.event_name,b.type,b.status,b.scheduled_start,c.selection_name,c.price,c.outcome
              FROM sports a
              INNER JOIN events b ON a.sports_id = b.sports_id
              INNER JOIN selections c ON c.event_id=b.event_id
              WHERE b.status=="started" AND b.type=="preplay"
              ''')
    event=c.fetchall()    
    for row in event:
        json_dict = {'sports_name': row[0], 'event_name ': row[1], 'type': row[2], 'status': row[3], 'scheduled_start': row[4],
                    'selection_name': row[5],'price': row[6],'outcome': row[7]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    s = json.loads(x)
    json_output = {'started': s}
    return json_output

# Method to get only winning selections
@app.route('/win', methods=['GET'])
def winning_selections():
    json_list = []              
    conn.row_factory = sqlite3.Row
    #cur = conn.cursor()
    c.execute('''
              SELECT a.event_name,a.type,a.actual_start,b.selection_name,b.price,b.outcome
              FROM events a
              INNER JOIN selections b ON a.event_id=b.event_id
              WHERE outcome=='win'
              ''')
    event=c.fetchall()    
    for row in event:
        json_dict = {'event_name': row[0], 'type ': row[1], 'actual_start': row[2], 'selection_name': row[3], 
                     'price': row[4],'outcome': row[5]}
        json_list.append(json_dict)
    x=json.dumps(json_list)
    s = json.loads(x)
    json_output = {'win': s}
    return json_output

#Main function
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Jan/2022 00:51:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:51:14] "GET /sports/football HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:51:22] "GET /sports HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:51:27] "POST /sports HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:51:30] "GET /events HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:51:34] "PUT /events HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:51:38] "GET /started HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:52:44] "PUT /events HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:52:50] "GET /started HTTP/1.1" 200 -
127.0.0.1 - - [26/Jan/2022 00:53:14] "GET /win HTTP/1.1" 200 -
